# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:

import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import datetime

In [3]:
%run -i 'sql_queries.py'
songplay_table_insert

'INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) VALUES (%s, %s, %s, %s,%s, %s, %s, %s)'

In [2]:
%run -i 'create_tables.py'



In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()



In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files("/home/workspace/data/song_data/")
song_files[0]

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [6]:
filepath = song_files[0]
filepath

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [7]:
df = pd.read_json(filepath, lines=True)
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id','title','artist_id','year','duration']]
song_data = (song_data.values)[0]
for i in song_data:
    print(type(i))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'float'>


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
## putting a bunch of songs into the songs table
## to delete
for song_file in song_files:
    filepath = song_file
    df1 = pd.read_json(filepath, lines=True)
    song_data = df1[['song_id','title','artist_id','year','duration']]
    song_data = (song_data.values)[0]
    #print(song_data)
    cur.execute(song_table_insert, song_data)
    conn.commit()
    
    

In [10]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_data = ((df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]).values)[0]
artist_data

array(['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan], dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [13]:
## adding all artists # to delete
song_files = get_files("/home/workspace/data/song_data/")
for song_file in song_files:
    filepath = song_file
    df1 = pd.read_json(filepath, lines=True)
    song_data = df1[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
    song_data = (song_data.values)[0]
    #print(song_data)
    cur.execute(artist_table_insert, song_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
logfiles = get_files("/home/workspace/data/log_data/")
logfiles

['/home/workspace/data/log_data/2018/11/2018-11-30-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-23-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-24-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-25-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-15-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-26-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-04-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.json',
 '/home/workspace/data/lo

In [15]:
filepath = logfiles[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [16]:
df =pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:
# checking possible events
(df['page']).unique()

array(['NextSong', 'Home', 'Logout', 'Settings', 'Downgrade', 'Login',
       'Help', 'Error', 'Upgrade'], dtype=object)

In [18]:
df = df.loc[df['page'] == "NextSong"]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [21]:
#newList = []
#for i in test:
#    newList.append(datetime.datetime.fromtimestamp(i))

# as list comprehension:
# [datetime.datetime.fromtimestamp(i) for i in (df['ts'])/1000]


In [20]:

t = pd.Series([datetime.datetime.fromtimestamp(i) for i in (df['ts'])/1000])
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
dtype: datetime64[ns]

In [21]:
time_df = pd.DataFrame({'start_time':t.dt.time,'hour':t.dt.hour,'day':t.dt.day,'week':t.dt.weekofyear,'month':t.dt.month,'year':t.dt.year,'weekday':t.dt.weekday_name})
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,00:22:07.796000,0,30,48,11,2018,Friday
1,01:08:41.796000,1,30,48,11,2018,Friday
2,01:12:48.796000,1,30,48,11,2018,Friday
3,01:17:05.796000,1,30,48,11,2018,Friday
4,01:20:56.796000,1,30,48,11,2018,Friday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [22]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [23]:
df = pd.read_json(filepath, lines=True)
df = df.loc[df['page'] == "NextSong"]
df = df[['userId','firstName','lastName','gender','level']]
user_df = df.rename(columns = {"userId":"user_id","firstName":"first_name","lastName":"last_name"})
user_df = user_df.drop_duplicates()
user_df.head()


,user_id,first_name,last_name,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
23,86,Aiden,Hess,M,free
30,24,Layla,Griffin,F,paid
40,26,Ryan,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [28]:
def sql(qry_string):
    cur.execute(qry_string)
    results = cur.fetchone()
    while results:
        print(results)
        results=cur.fetchone()

In [30]:
sql("select * from artists")

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan)
('ARNTLGG11E2835DDB9', 'Clp', '', nan, nan)
('AR8ZCNI1187B9A069B', 'Planet P Project', '', nan, nan)
('AR10USD1187B99F3F1', 'Tweeterfriendly Music', 'Burlington, Ontario, Canada', nan, nan)
('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892)
('ARKFYS91187B98E58F', 'Jeff And Sheri Easter', '', nan, nan)
('ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan)
('ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', 35.21962, -80.01955)
('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan)
('ARXR32B1187FB57099', 'Gob', '', nan, nan)
('ARKRRTF1187B9984DA', 'Sonora Santanera', '', nan, nan)
('ARIG6O41187B988BDD', 'Richard Souther', 'United States', 37.16793, -95.84502)
('AR1ZHYZ1187FB3C717', 'Faiz Ali Faiz', '', nan, nan)
('ARD842G1187B997376', 'Blue Rodeo', 'Toronto, Ontario, Canada', 43.64856, -79.38533)
('ARL7K851187B99ACD2', 'Andy Andy', '', nan, nan)
('ARHHO3O1187B989413', 'Bob Azzam', '', nan, n

In [25]:
cur.execute("select *from (songs join artists on songs.artist_id = artists.artist_id) where (name = 'Casual' and duration = 218.93179)")

row=cur.fetchone()
while row:
    print(row)
    row=cur.fetchone()

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179, 'ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan)


In [27]:
test = ''
cur.execute(song_select, ('Casual', 218.93179))
row=cur.fetchone()
while row:
    print(row)
    row=cur.fetchone()

('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1')
('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1')
('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1')


In [28]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [29]:
## adding all users to table # to delete
for logfile in logfiles:
    df1 = pd.read_json(logfile, lines=True)
    df1 = df1.loc[df1['page'] == "NextSong"]
    df1 = df1[['userId','firstName','lastName','gender','level']]
    user_df = df1.rename(columns = {"userId":"user_id","firstName":"first_name","lastName":"last_name"})
    user_df = user_df.drop_duplicates()
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)
        conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [30]:
df = pd.read_json(filepath, lines=True)
song_plays = df[['ts','userId','level','artist','song','length','sessionId','location','userAgent']]
song_plays.head()

,ts,userId,level,artist,song,length,sessionId,location,userAgent
0,1543537327796,91,free,Stephen Lynch,Jim Henson's Dead,182.85669,829,"Dallas-Fort Worth-Arlington, TX",Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...
1,1543540121796,73,paid,Manowar,Shell Shock,247.56200,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,1543540368796,73,paid,Morcheeba,Women Lose Weight (Feat: Slick Rick),257.41016,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,1543540625796,73,paid,Maroon 5,Won't Go Home Without You,231.23546,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
4,1543540856796,73,paid,Train,Hey_ Soul Sister,216.76363,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


In [31]:
## all song_plays
#logfiles = get_files("/home/workspace/data/log_data/")
#for logfile in logfiles:
#    df = pd.read_json(logfile, lines=True)
#    song_plays = df[['ts','userId','level','artist','song','length','sessionId','location','userAgent']]
   # print(song_plays)


In [32]:
#cur.execute(song_select, ('Casual', 218.93179))
#results = cur.fetchone()
#while results:
#    print(results)
#    results=cur.fetchone()


In [37]:
for index, row in song_plays.iterrows():

    # get songid and artistid from song and artist tables
    #cur.execute(song_select, (row.song, row.artist, row.length))
    cur.execute(song_select, (row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
     
    
    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    #print(songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    #conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [34]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

## OPTIONAL: Question for the reviewer
 
If you have any question about the starter code or your own implementation, please add it in the cell below. 

For example, if you want to know why a piece of code is written the way it is, or its function, or alternative ways of implementing the same functionality, or if you want to get feedback on a specific part of your code or get feedback on things you tried but did not work.

Please keep your questions succinct and clear to help the reviewer answer them satisfactorily.  

> **_Your question_**